In [1]:
from pathlib import Path

funcs_csv = Path('/home/cls0027/exp_builds/astera.exp/rundata/run1/0.fighter/functions.csv')
locals_csv = Path('/home/cls0027/exp_builds/astera.exp/rundata/run1/0.fighter/locals.csv')

import pandas as pd
pd.set_option('display.max_colwidth', 20)   # JSON columns make things look weird in notebook without this


funcs_df = pd.read_csv(funcs_csv)
locals_df = pd.read_csv(locals_csv)

In [2]:
from astlib import StructDatabase
#!ls ~/exp_builds/astera.exp/rundata/run1/0.fighter/0.fighter.debug.sdb

debug_sdb_file = Path.home()/'exp_builds/astera.exp/rundata/run1/0.fighter/0.fighter.debug.sdb'
sdb = StructDatabase.from_json(debug_sdb_file)

In [3]:
sid = sdb.sids_by_name['r_camera'][0]
sdb.structs_by_id[sid]

struct r_camera {
	0x0: float[3] position
	0xc: float[2] size
	0x14: float rotation
	0x18: float[4][4] view
	0x58: float[4][4] projection
	0x98: float near
	0x9c: float far
}

In [4]:
locals_df.loc[locals_df.TypeSeq_Debug.apply(lambda x: 'STRUCT' in x)].groupby('TypeSeq_Debug').count()

,FunctionStart,Signature,Name_Strip,Type_Strip,LocType_Strip,LocRegName_Strip,LocOffset_Strip,TypeCategory_Strip,TypeSeq_Strip,Name_Debug,Type_Debug,LocType_Debug,LocRegName_Debug,LocOffset_Debug,TypeCategory_Debug,HasDWARF,TypeJson_Debug,BinaryId
TypeSeq_Debug,,,,,,,,,,,,,,,,,,
"ARR,STRUCT",4,4,4,4,4,0,4,4,4,4,4,4,0,4,4,4,4,4
"PTR,PTR,STRUCT",10,10,10,10,10,4,10,10,10,10,10,10,4,10,10,10,10,10
"PTR,STRUCT",444,444,444,444,444,242,444,444,444,444,444,444,242,444,444,444,444,444
STRUCT,59,59,59,59,59,0,59,59,59,59,59,59,0,59,59,59,59,59


In [5]:
# start with PTR->STRUCT, this is likely the easiest

var_row = locals_df[locals_df.TypeSeq_Debug=='PTR,STRUCT'].iloc[0]
var_row

FunctionStart                     1325033
Signature                        27,64,89
Name_Strip                          lVar2
Type_Strip                          int64
LocType_Strip                    register
LocRegName_Strip                      rax
LocOffset_Strip                         0
TypeCategory_Strip                BUILTIN
TypeSeq_Strip                       int64
Name_Debug                          asset
Type_Debug                       asset_t*
LocType_Debug                    register
LocRegName_Debug                      rax
LocOffset_Debug                       0.0
TypeCategory_Debug                    PTR
TypeSeq_Debug                  PTR,STRUCT
HasDWARF                            False
TypeJson_Debug        {"kind": "Pointe...
BinaryId                                0
Name: 16, dtype: object

Read datatype from JSON, since we have the struct db we have the layout:

In [19]:
import astlib
from varlib.datatype import datatype_from_json_str

dt = datatype_from_json_str(var_row.TypeJson_Debug, sdb)
dt.pointed_to.layout

0x0: uint32 uid
0x8: uchar* data
0x10: uint32 data_length
0x18: char* name
0x20: int32 chunk_start
0x24: int32 chunk_length
0x28: uchar fs
0x29: uchar filled
0x2a: uchar req
0x2b: uchar req_free
0x2c: uchar chunk

Ok, so let's collect all the member accesses from the Debug AST for this variable

In [33]:
var_name = var_row.Name_Debug
func_addr = var_row.FunctionStart
bin_id = var_row.BinaryId
print(f'Binary: {bin_id}, func {func_addr:#x}, local var "{var_name}"end=')

Binary: 0, func 0x1437e9, local var "asset"


In [47]:
debug_ast_jsonfile = funcs_df[funcs_df.FunctionStart==func_addr].iloc[0].AstJson_Debug
ast = astlib.read_json(debug_ast_jsonfile, sdb)

**pick up here**

In [73]:
# --------------------------------------------------
# TODO: pick up here and VISIT the AST to collect all MemberExpr
# nodes that involve our target variable
# --------------------------------------------------
# --> or collect ALL MemberExprs and try to map each of them to StripAST

# NOTE: see initial start on implementation in basic_dataset for reference...

vdecl = ast.inner[-1].inner[-1].inner[1].inner[0]
vdecl.dtype.pointed_to

struct asset_t {
	0x0: uint32 uid
	0x8: uchar* data
	0x10: uint32 data_length
	0x18: char* name
	0x20: int32 chunk_start
	0x24: int32 chunk_length
	0x28: uchar fs
	0x29: uchar filled
	0x2a: uchar req
	0x2b: uchar req_free
	0x2c: uchar chunk
}

In [16]:
#!ls ~/exp_builds/astera.exp/rundata/run1/0.fighter/*.sdb

/home/cls0027/exp_builds/astera.exp/rundata/run1/0.fighter/0.fighter.debug.sdb
/home/cls0027/exp_builds/astera.exp/rundata/run1/0.fighter/0.fighter.sdb
/home/cls0027/exp_builds/astera.exp/rundata/run1/0.fighter/fighter.dwarf.sdb
